In [20]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from helpers import get_abspath

In [44]:
oaName = 'RHC'
pardir = 'results\\NN'
resfile = 'results.csv'
res = get_abspath('{}\\{}'.format(oaName, resfile), pardir)
df = pd.read_csv(res)

In [45]:
# get columns
iterations = df['iteration']
MSE_train = df['MSE_train']
MSE_test = df['MSE_test']
MSE_valid =df['MSE_validation']

# create error curve
plt.figure(0)
plt.plot(iterations, MSE_train, color='b', label='Training')
plt.plot(iterations, MSE_test, color='r', label='Validation')
plt.plot(iterations, MSE_valid, color='g', label='Test')
plt.legend(loc='best')
plt.grid(linestyle='dotted')
plt.xlabel('Iterations')
plt.ylabel('Mean squared error')
# plt.ylim((0, 1))

# save learning curve plot as PNG
plotdir = 'plots\\NN'
plot_tgt = '{}\\{}'.format(plotdir, oaName)
plotpath = get_abspath('{}_error.png'.format(oaName), plot_tgt)
plt.savefig(plotpath, bbox_inches='tight')
plt.close()